In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.metrics import pairwise

In [2]:
# Create Global Variables
background1 = None
background2 = None
accumulated_weight1 = 0.5
accumulated_weight2 =0.5
# roi = region of interest
roi_top = 200
roi_bottom = 450
roi_right = 370
roi_left = 620
arearatio = 0 
expressions = []
text = ''
operators = ['+','-','*','/']
answer = None
thumb = False
new = '0'
new_op = ''
old_op = ''

In [3]:
# Create the Clear Function
def clear():
    """
    This acts as the clear function on a calculator
    reseting all the variables
    """
    global expressions, text, new, new_op, old_op, answer
    expressions = []
    text = ''
    new = '0'
    new_op = ''
    old_op = ''
    answer = None
    # return (expressions,text,new,new_op,old_op,answer)

In [4]:
# Create the background detection
def background_identification1(frame):
    """
    Arguments:
    frame:
    
    """
    global background1
    if background1 is None:
        background1 = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background1,accumulated_weight1)

In [5]:
# Do the background again
def background_identification2(frame):
    """
    Arguments:
    frame:
    weight
    """
    global background2
    if background2 is None:
        background2 = frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background2,accumulated_weight2)

In [6]:
# Segment the hand
def segment(frame,background,threshold_min = 30):
    """
    Arguments:
    frame:
    background:
    threshhold_min:
    
    """
    diff = cv2.absdiff(background.astype('uint8'),frame)
    ret,thresholded = cv2.threshold(diff,threshold_min,255,cv2.THRESH_BINARY)
    kernel = np.ones((3,3),np.uint8)
    thresholded = cv2.morphologyEx(thresholded,cv2.MORPH_CLOSE,kernel)
    contours,hierarchy = cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    
    if len(contours) == 0:
        return None
    else:
        hand_segment = max(contours,key=cv2.contourArea)
        return (thresholded,hand_segment)

In [7]:
def expression_selection(expressions, old_op, text):
    """
    Arguments:
    expressions: list of all the expressions "+, -, /, *, ="
    old_op: The old operation that was previously "clicked" 
    text: The calculator text that shows current operation and numbers

    Function allows the user to select the operation that they would
    like to perform. Using the finger_segment we can locate the users
    finger that is outside of the box and update x and y coordinates. 
    The x and y are used to diside which operation the user is selecting.

    return: This returns the "calculator" text
    """
    global first_input, new_op, new, answer, finger 
    __ , finger_segment = finger
    conv_hull2 = cv2.convexHull(finger_segment)
    point = tuple(conv_hull2[conv_hull2[:, :, 1].argmin()][0])
    x = point[0] 
    y = point[1]
    cv2.circle(roi2,(x,y),3,(50,20,0), -1)
    
    if((num_frames%50 == 0) and (len(expressions)!=0) and (expressions[-1] not in operators)):
        if((x>0 and x<=80) and (y>0 and y<40)):
            cv2.putText(frame_copy,'+',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '+'
            if((old_op != new_op) and (len(expressions)!=0)):
                expressions.append(new_op)
        if((x>80 and x<=160) and (y>0 and y<40)):
            cv2.putText(frame_copy,'-',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '-'
            if((old_op != new_op) and len(expressions)!=0):
                expressions.append(new_op)
        if((x>160 and x<=240) and (y>0 and y<40)):
            cv2.putText(frame_copy,'*',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '*'
            if((old_op != new_op) and len(expressions)!=0):
                expressions.append(new_op)
        if((x>240 and x<=320) and (y>0 and y<40)):
            cv2.putText(frame_copy,'/',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '/'
            if((old_op != new_op) and len(expressions)!=0):
                expressions.append(new_op)
        if((x>320 and x<=400) and (y>0 and y<40)):
            cv2.putText(frame_copy,'=',(100,100), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
            new_op = '='
            answer = eval(text)
        if((x>320 and x<=400) and (y>80 and y<120)):
            clear()

    # Update variables
    first_input = True
    old_op = new_op
    return text
            


In [14]:
cam = cv2.VideoCapture(0)
num_frames = 0
first_input = True
while True:
    ret,frame = cam.read()
    frame_copy = frame.copy()
    frame_copy=cv2.flip(frame_copy,1)
    roi = frame_copy[roi_top:roi_bottom,roi_right:roi_left]
    gray = cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray,(5,5),100)
    
    # Placing text inside the box to match the associated expression
    cv2.putText(frame_copy,'+',(50,50), cv2.FONT_HERSHEY_SIMPLEX,1,(204, 0, 153),2)
    cv2.putText(frame_copy,'-',(130,50), cv2.FONT_HERSHEY_SIMPLEX,1,(204, 0, 153),2)
    cv2.putText(frame_copy,'*',(210,50), cv2.FONT_HERSHEY_SIMPLEX,1,(204, 0, 153),2)
    cv2.putText(frame_copy,'/',(290,50), cv2.FONT_HERSHEY_SIMPLEX,1,(204, 0, 153),2)
    cv2.putText(frame_copy,'=',(368,50), cv2.FONT_HERSHEY_SIMPLEX,1,(204, 0, 153),2)
    cv2.putText(frame_copy,'C ',(370,130), cv2.FONT_HERSHEY_SIMPLEX,1,(204, 0, 153),2)
    


    roi2 = frame_copy[20:140,20:420]
    op_gray = cv2.cvtColor(roi2,cv2.COLOR_BGR2GRAY)
    op_gray = cv2.GaussianBlur(op_gray,(5,5),100)

    if num_frames<200:
        background_identification1(gray)
        background_identification2(op_gray)
        cv2.putText(frame_copy,"DOING MAGIC, DONT MOVE",(300,500),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
        cv2.imshow('Finger Count',frame_copy)  

    else:
        hand = segment(gray,background1)
        if hand is not None:
            cv2.imshow('Finger Count',frame_copy)
            thresholded , hand_segment = hand
            conv_hull =cv2.convexHull(hand_segment)
            areacont = cv2.contourArea(hand_segment)
            areahull = cv2.contourArea(conv_hull)
            if(areacont != 0):
                arearatio=((areahull-areacont)/areacont)*100
            
            epsilon = 0.0005*cv2.arcLength(hand_segment,True)
            approx= cv2.approxPolyDP(hand_segment,epsilon,True)
            
            conv_hull =cv2.convexHull(approx,returnPoints =False)
            defects = None
            try:
                defects = cv2.convexityDefects(approx,conv_hull)
            except:
                pass
            if defects is not None:
                l=0
                thumb = False
                for i in range(defects.shape[0]):
                    s,e,f,d = defects[i,0]
                    start = tuple(approx[s][0])
                    end = tuple(approx[e][0])
                    far = tuple(approx[f][0])
                    pt= (100,180)


                    # find length of all sides of triangle
                    a = math.sqrt((end[0] - start[0])**2 + (end[1] - start[1])**2)
                    b = math.sqrt((far[0] - start[0])**2 + (far[1] - start[1])**2)
                    c = math.sqrt((end[0] - far[0])**2 + (end[1] - far[1])**2)
                    s = (a+b+c)/2
                    ar = math.sqrt(s*(s-a)*(s-b)*(s-c))

                    #distance between point and convex hull
                    d=(2*ar)/a

                    # apply cosine rule here
                    angle = math.acos((b**2 + c**2 - a**2)/(2*b*c)) * 57


                    # ignore angles > 90 and ignore points very close to convex hull(they generally come due to noise)
                    if angle <= 100 and d>30:
                        l += 1
                        cv2.circle(roi, far, 3, (255,0,0), -1)
                        if(angle>50):
                            thumb = True

                    #draw lines around hand
                    cv2.line(roi,start, end, (0,255,0), 2)

                l+=1
                if(num_frames%50 == 0):
                    # if detection of l == 1 will either detect 1(when starts with pointer finger) or will detect 6(when starts with thumb)
                    if(l==1):
                        if(arearatio<12 and (first_input == False)):
                            cv2.putText(frame_copy,'0',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '0'
                            expressions.append(new)
                        elif(arearatio<17.5):
                            cv2.putText(frame_copy,'6',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '6'
                            expressions.append(new)
                        else:
                            first_input = False
                            cv2.putText(frame_copy,'1',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '1'
                            expressions.append(new)

                    # if detection of l == 2 will either detect 2(when starts with pointer finger) or will detect 7(when starts with thumb)
                    if(l==2):
                        first_input = False
                        if(thumb == True):
                            cv2.putText(frame_copy,'7',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '7'
                        else:   
                            cv2.putText(frame_copy,'2',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '2'
                        expressions.append(new)
                    # if detection of l == 3 will either detect 3(when starts with pointer finger) or will detect 8(when starts with thumb)
                    if(l==3):
                        first_input = False
                        if(thumb == True):
                            cv2.putText(frame_copy,'8',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '8'
                        else:
                            cv2.putText(frame_copy,'3',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)  
                            new = '3'
                        expressions.append(new)
                    # if detection of l == 4 will either detect 4(when starts with pointer finger) or will detect 9(when starts with thumb)
                    if(l==4):
                        first_input = False
                        if(thumb == True):
                            cv2.putText(frame_copy,'9',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '9'
                        else:
                            cv2.putText(frame_copy,'4',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                            new = '4'
                        expressions.append(new)
                    # if detection of l == 1 will detect 5 (when hand is open flat)
                    if(l==5):
                        first_input = False
                        cv2.putText(frame_copy,'5',(20,20), cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,0),2)
                        new = '5'
                        expressions.append(new)
                cv2.imshow('Thresholded',thresholded)
        text = ''
        for op in expressions: 
            text += op 
        else:
            cv2.imshow('Finger Count',frame_copy)
            finger = segment(op_gray,background2)
            # Find the the location of the finger that is clicking the expressions
            # one this is done then the finger is followed by the the point and
            # If the location of the point is within the x and y of one of the expressions we 
            # Then perform the expression that is "clicked"
            if finger is not None:
                
                text = expression_selection(expressions, old_op, text)

                
                        

    if(answer is not None):
        cv2.putText(frame_copy,"="+str(answer),(50,175), cv2.FONT_HERSHEY_SIMPLEX,1,(77, 230, 0),2)
    cv2.putText(frame_copy,text,(50,125), cv2.FONT_HERSHEY_SIMPLEX,1,(77, 230, 0),2)
    cv2.rectangle(frame_copy,(roi_left,roi_top),(roi_right,roi_bottom),(102, 51, 0),5)
    num_frames += 1        
    cv2.imshow('Finger Count',frame_copy)
        
    k = cv2.waitKey(1) & 0xFF
    
    if k == 27:
        break
        
cam.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 